<a href="https://colab.research.google.com/github/rahulkashyap14298/AI-Project/blob/master/Copy_of_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extracting MFCC's For every audio file**

 Importing the important libraries. Reading the csv file and creating the file path


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [ ]:

from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('/content/gdrive/MyDrive/Audio-Classification-master.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()


In [ ]:
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm


In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
import os
import IPython.display as ipd
extracted_features=[]
directory ="/content/Audio-Classification-master/wavfiles"
for files in os.listdir(directory):
  for audio in os.listdir(directory+"/"+files):
    final_class_labels=files
    data=features_extractor(directory+"/"+files+"/"+audio)
    extracted_features.append([data,final_class_labels])
    


    
    
    


In [ ]:
print(extracted_features)



[[array([-395.43433  ,  -42.438892 ,  -32.611206 ,   26.436132 ,
        -12.596503 ,   -7.3042254,  -29.151024 ,    7.054409 ,
        -16.607744 ,   13.6080265,    6.803552 ,   26.704388 ,
         -2.9316394,   29.533422 ,   12.399391 ,   10.099918 ,
        -14.980867 ,   -7.102765 ,    2.2680883,   21.175205 ,
        -11.832846 ,   -7.1997333,  -16.43158  ,   -7.4697604,
         -1.2796584,    7.2820563,   -5.87565  ,    1.904466 ,
          5.2456093,    5.956822 ,  -14.035757 ,  -17.352228 ,
         -9.076789 ,   11.121372 ,   -1.0366164,   -5.561257 ,
         -1.5791967,    1.5150236,   -5.1248007,   -1.9046451],
      dtype=float32), 'Hi_hat'], [array([-4.1065689e+02, -5.0238140e+01, -6.4130135e+01,  1.1438326e+01,
       -2.1625063e+01,  7.0340314e+00, -2.6496962e+01,  4.9762983e+00,
       -2.3479002e+01,  5.2538319e+00, -8.9219456e+00,  6.7080078e+00,
       -1.8546154e+01,  5.1096220e+00, -1.6872828e+01,  1.0810004e+00,
       -6.1116681e+00,  1.0462702e+01, -3.4519792

Creating the feature extractor function. Extracting the mfcc features from the discrete data points of a single audio file and then scaling it

Now we iterate through every audio file and extract features using Mel-Frequency Cepstral Coefficients

Converting extracted_features to Pandas dataframe

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])

Split the dataset into independent and dependent dataset. Creating the dataset on which the model will undergo training

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

 **TRAINING THE MODEL**

Label Encoding

In [ ]:
y=np.array(pd.get_dummies(y))

TRAIN TEST SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

MODEL CREATION 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

num_labels=y.shape[1]



model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
print(y.shape[0])

300


In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

Training my model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 1000
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/1000
8/8 [==============================] - 0s 16ms/step - loss: 1.6684 - accuracy: 0.4167 - val_loss: 1.6265 - val_accuracy: 0.4333

Epoch 00001: val_loss improved from inf to 1.62645, saving model to saved_models/audio_classification.hdf5
Epoch 2/1000
8/8 [==============================] - 0s 8ms/step - loss: 1.5649 - accuracy: 0.4333 - val_loss: 1.6503 - val_accuracy: 0.5000

Epoch 00002: val_loss did not improve from 1.62645
Epoch 3/1000
8/8 [==============================] - 0s 8ms/step - loss: 1.6126 - accuracy: 0.4167 - val_loss: 1.6655 - val_accuracy: 0.4833

Epoch 00003: val_loss did not improve from 1.62645
Epoch 4/1000
8/8 [==============================] - 0s 9ms/step - loss: 1.5093 - accuracy: 0.4417 - val_loss: 1.6335 - val_accuracy: 0.5167

Epoch 00004: val_loss did not improve from 1.62645
Epoch 5/1000
8/8 [==============================] - 0s 9ms/step - loss: 1.5375 - accuracy: 0.4500 - val_loss: 1.5973 - val_accuracy: 0.4667

Epoch 00005: val_loss improved fro

Predicting for the test set

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.6499999761581421


Predicting for the single audio file

In [ ]:
filename="/content/Audio-Classification-master/wavfiles/Acoustic_guitar/160c92d7.wav"
prediction_feature=features_extractor(filename)
prediction_feature=prediction_feature.reshape(1,-1)
np.argmax(model.predict(prediction_feature), axis=-1)

[0]


NameError: ignored